In [10]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
data_train = np.load('data/pixelized.npz')
data_labels = np.load('data/pixelized_class.npz')

In [12]:
x_train = data_train['arr_0'][:3000]
x_labels = data_labels['arr_0'][:3000]


In [13]:
category_mapping = {90: 'SN1a', 67: 'SN1a-91bg', 52: 'SN1ax', 42: 'SN2', 62: 'SN1bc', 95: 'SLSN1', 15: 'TDE', 64: 'KN', 88: 'AGN', 92: 'RRL', 65: 'M-dwarf', 16: 'EB', 53: 'Mira', 6: 'Microlens'}

In [14]:
y_train = []
for i,obj_class in enumerate(x_labels):
    #class_ = meta_[meta_['object_id'] == obj_id]['target'].values[0]
    
    if category_mapping[obj_class] == 'AGN':
        y_train.append(1)
    else:
        y_train.append(0)
    
y_train = np.array(y_train)

In [15]:
# Model / data parameters
num_classes = 2
input_shape = (256, 1300, 6)

y_train = keras.utils.to_categorical(y_train, num_classes)

In [16]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(6, 6), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 251, 1295, 32)     6944      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 125, 647, 32)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 123, 645, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 61, 322, 64)      0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1257088)           0         
                                                                 
 dropout_1 (Dropout)         (None, 1257088)          

In [ ]:
batch_size = 32
epochs = 10

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_img.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=2,
        verbose=1,
        mode="max",
        restore_best_weights=True
    ),
]

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks)


"""
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)
"""

Epoch 1/10
10/85 [==>...........................] - ETA: 2:56 - loss: 0.1614 - accuracy: 0.9625